
# Prediksi Hilal
---

In [1]:
# _jengkolrebus
# Mei, 2020
# Curup, Bengkulu

Program ini menggunakan Python Package: [Skyfield](https://rhodesmill.org/skyfield/).
Program ini dibuat dengan ketentuan sebagai berikut:
- Ephemeris:
    > Ephemeris yang digunakan adalah de421.bsp
- Definisi Matahari Terbenam:
    > Skyfield menggunakan definisi resmi matahari terbit dan terbenam dari United States Naval Observatory, yang mendefinisikan matahari terbenam ketika pusat matahari adalah <b>0,8333</b> derajat di bawah Horizon, untuk memperhitungkan rata-rata jari-jari Matahari dan untuk rata-rata pembiasan atmosfer di horizon.
___
- Flowchart:
<br>
<img src="flowchart.png">
<br>
- Input Lokasi berupa Latitude dan Longitude dalam bentuk <em>Decimal Degree</em> dengan format String / text.
 > Contoh : 
    Latitude: "7.83305556 S"
    Longitude: "110.38305556 E"
    Perhatikan Notasi "S" dan "E" bergantung pada lokasi.
    
- Input waktu berupa tanggal rentang waktu hilal yang ingin dicari.
 > Contoh dari tanggal 1 Januari 2020, sampai 1 Januari 2021
___


In [2]:
from skyfield.api import load, Topos
from skyfield.units import Angle
from skyfield import almanac
from datetime import datetime, timedelta
from pytz import timezone
from ipywidgets import widgets, interact, interactive
from IPython.display import display, HTML
import pandas as pd
import calendar

In [3]:
class var:
    df = []

In [4]:
jkt = timezone('Asia/Jakarta')
ts = load.timescale()
e = load('de421.bsp')

### Kriteria Hilal
- Imkanur Rukyat
    > - Pada saat matahari terbenam, <b>ketinggian bulan minimal 2$^\circ$</b>, <b>Elongasi minimal 3$^\circ$</b>.
    > - Usia bulan minimal <b>8 Jam</b>, dihitung sejak ijtima' (konjungsi)
    
___

In [5]:
class Find():
    def __init__(self, lat, long, t0, t1):
        self.lat = lat
        self.long = long
        self.t0 = t0
        self.t1 = t1
        self.topo = Topos(self.lat, self.long)
        self.loc = e['earth'] + self.topo
    
    def conjunction(self):
        result = []
        t0 = ts.utc(self.t0)
        t1 = ts.utc(self.t1)
        f = almanac.oppositions_conjunctions(e, e['moon'])
        t, y = almanac.find_discrete(t0, t1, f)
        for ti, yi in zip(t, y):
            if(yi == 1):
                result.append(ti)
            else:
                pass

        return result
        
    def sunset(self, t):
        t = t.utc
        t0 = ts.utc(t[0], t[1], t[2], t[3], t[4], t[5])
        t1 = ts.utc(t[0], t[1], t[2]+1, t[3], t[4], t[5])
        f = almanac.sunrise_sunset(e, self.topo)
        t, y = almanac.find_discrete(t0, t1, f)
        for ti, yi in zip(t, y):
            if(yi == False):
                return ti
            else:
                pass
    
    def objPos(self, t, obj):
        astrometric = self.loc.at(t).observe(e[obj])
        alt, az, d = astrometric.apparent().altaz()
        return alt, az, astrometric
        


In [6]:
# Metode untuk membandingkan dengan Imkan Rukyat
def imkanRukyat(alt, elong, age):
    if(alt.degrees >= 2 and elong.degrees >= 3 and (timedelta.total_seconds(age)/3600) > 8):
        return u'\u2714'
    else:
        return u'\u2718'

def result(lat, long, t0, t1):
    f = Find(lat, long, t0, t1)
    conj = f.conjunction()
    sunset = [f.sunset(t) for t in conj]
    
    moon_alt = []
    moon_az = []
    moon_astrometric = []
    sun_alt = []
    sun_az = []
    sun_astrometric = []
    for t in sunset:
        alt, az, astro = f.objPos(t, 'moon')
        moon_alt.append(alt)
        moon_az.append(az)
        moon_astrometric.append(astro)
        
        alt, az, astro = f.objPos(t, 'sun')
        sun_alt.append(alt)
        sun_az.append(az)
        sun_astrometric.append(astro)
    
    elong = [moon.separation_from(sun) for moon, sun in zip(moon_astrometric, sun_astrometric)]
        
    conj[:] = [t.astimezone(jkt).replace(tzinfo=None) for t in conj]
    sunset[:] = [t.astimezone(jkt).replace(tzinfo=None) for t in sunset]
    
    moon_age = [t1-t0 for (t0, t1) in zip(conj, sunset)]
    imkan_rukyat = [imkanRukyat(al, el, age) for al, el, age in zip(moon_alt, elong, moon_age)]
    
    # Menampilkan hasil dalam bentuk tabel dataframe
    tabel = list(zip(conj, sunset,
                     moon_alt, moon_az, 
                     sun_alt, sun_az, 
                     elong, moon_age,
                    imkan_rukyat))
    
    df = pd.DataFrame(tabel, columns=['Waktu Konjungsi (UTC+07)', 'Waktu Hilal (UTC+07)', 
                                      'Altitude Bulan', 'Azimuth Bulan', 
                                      'Altitude Matahari', 'Azimuth Matahari', 
                                     'Elongasi', 'Usia Bulan', 
                                     'Imkan Rukyat'])
    df.index+=1
    display(df)
    
    var.df = df
    
    button = widgets.Button(description="Save to Excel")
    output = widgets.Output()
    display(button, output)
    def on_button_clicked(b):
        var.df.to_excel("output.xlsx")
        with output:
            print("Button clicked.")

    button.on_click(on_button_clicked)

In [7]:
hari_ini = datetime.now().date()
days_in_month = calendar.monthrange(hari_ini.year, hari_ini.month)[1]
bulan_depan = hari_ini + timedelta(days_in_month)

dari = widgets.DatePicker(value=hari_ini,
    description='Dari:',
    disabled=False
)
sampai = widgets.DatePicker(value=bulan_depan,
    description='Sampai:',
    disabled=False
)

lat = widgets.Text(value='7.83305556 S', description='Latitude:')
long = widgets.Text(value='110.38305556 E', description='Longitude:')

interactive(result, lat=lat, long=long, t0=dari, t1=sampai)


interactive(children=(Text(value='7.83305556 S', description='Latitude:'), Text(value='110.38305556 E', descri…